In [18]:
import pandas as pd
import numpy as np

In [9]:
pre_assembled_file = 'Input/Processed/osha_data.csv'
database = pd.read_csv(pre_assembled_file, index_col=0)

C:\Users\JKVASNIC\AppData\Local\Temp\ipykernel_17168\2146351305.py:2: DtypeWarning: Columns (1,6,10,11,19) have mixed types. Specify dtype option on import or set low_memory=False.
  database = pd.read_csv(pre_assembled_file, index_col=0)


In [10]:
# Assuming 'database' is a pandas DataFrame loaded previously

# Numeric conversions with coercion for non-numeric values
database['AIR_VOLUME_SAMPLED'] = pd.to_numeric(database['AIR_VOLUME_SAMPLED'], errors='coerce')

# Factor conversion with specified levels
database['BLANK_USED'] = pd.Categorical(database['BLANK_USED'], categories=['Y', 'N'])

# Ensure string type
database['CITY'] = database['CITY'].astype(str)

# Lowercase and date conversion handling multiple formats
def convert_date(column):
    return pd.to_datetime(
        column.str.lower(),
        errors='coerce',
        format='%Y-%b-%d'
    ).combine_first(
        pd.to_datetime(column.str.lower(), errors='coerce', format='%Y/%m/%d')
    ).combine_first(
        pd.to_datetime(column.str.lower(), errors='coerce', format='%Y-%m-%d')
    )

database['DATE_REPORTED'] = convert_date(database['DATE_REPORTED'])
database['DATE_SAMPLED'] = convert_date(database['DATE_SAMPLED'])

# Categorical data conversion with specified levels
database['EIGHT_HOUR_TWA_CALC'] = pd.Categorical(database['EIGHT_HOUR_TWA_CALC'], categories=['Y', 'N'])

# Ensure string type
database['ESTABLISHMENT_NAME'] = database['ESTABLISHMENT_NAME'].astype(str)
database['FIELD_NUMBER'] = database['FIELD_NUMBER'].astype(str)

# Formatting and factor conversion for codes
database['IMIS_SUBSTANCE_CODE'] = pd.Categorical(
    database['IMIS_SUBSTANCE_CODE'].str.replace(' ', '0').str.zfill(4)
)
database['INSPECTION_NUMBER'] = pd.Categorical(database['INSPECTION_NUMBER'])

database['INSTRUMENT_TYPE'] = database['INSTRUMENT_TYPE'].astype(str)
database['LAB_NUMBER'] = pd.Categorical(database['LAB_NUMBER'])

# NAICS_CODE cleaning based on length condition
database['NAICS_CODE'] = database['NAICS_CODE'].astype(str).apply(lambda x: x if len(x) >= 6 else np.nan)

database['OFFICE_ID'] = pd.Categorical(database['OFFICE_ID'])
database['QUALIFIER'] = database['QUALIFIER'].astype(str)

# Numeric conversion with coercion
database['SAMPLE_RESULT'] = pd.to_numeric(database['SAMPLE_RESULT'], errors='coerce')

# Categorical conversion without specified levels here as not provided in the snippet
database['SAMPLE_TYPE'] = pd.Categorical(database['SAMPLE_TYPE'])
database['SAMPLE_WEIGHT'] = pd.to_numeric(database['SAMPLE_WEIGHT'], errors='coerce')
database['SAMPLING_NUMBER'] = pd.Categorical(database['SAMPLING_NUMBER'])
database['SIC_CODE'] = pd.Categorical(database['SIC_CODE'])
database['STATE'] = pd.Categorical(database['STATE'])
database['SUBSTANCE'] = database['SUBSTANCE'].astype(str)

# Numeric conversion with coercion
database['TIME_SAMPLED'] = pd.to_numeric(database['TIME_SAMPLED'], errors='coerce')
database['UNIT_OF_MEASUREMENT'] = database['UNIT_OF_MEASUREMENT'].astype(str)

# Formatting and factor conversion for ZIP_CODE
database['ZIP_CODE'] = database['ZIP_CODE'].astype(str).str.replace(' ', '0').str.zfill(5)
database['ZIP_CODE'] = pd.Categorical(database['ZIP_CODE'])

# Extracting the year from a date and converting to categorical
database['YEAR'] = pd.Categorical(database['DATE_SAMPLED'].dt.year)

In [21]:
r_database = pd.read_parquet('database.parquet')

r_database.equals(database)

False

In [26]:
# Check if both dataframes are identical
are_identical = database.equals(r_database)
print("Are the two dataframes identical? ", are_identical)

# Function to compare dataframes column by column
def compare_dataframes(df1, df2):
    diff_columns = []
    for column in df1.columns:
        if column in df2.columns:
            if not df1[column].equals(df2[column]):
                print(f"Column {column} differs.")
                diff_columns.append(column)
        else:
            print(f"Column {column} is not present in both dataframes.")
    return diff_columns

diff_columns = compare_dataframes(database, r_database)

Are the two dataframes identical?  False
Column DATE_REPORTED differs.
Column DATE_SAMPLED differs.
Column ESTABLISHMENT_NAME differs.
Column FIELD_NUMBER differs.
Column INSPECTION_NUMBER differs.
Column NAICS_CODE differs.
Column OFFICE_ID differs.
Column SAMPLE_RESULT differs.
Column SAMPLING_NUMBER differs.
Column SIC_CODE differs.
Column STATE differs.
Column SUBSTANCE differs.
Column YEAR differs.
Column ZIP_CODE differs.


In [27]:
diff_columns

['DATE_REPORTED',
 'DATE_SAMPLED',
 'ESTABLISHMENT_NAME',
 'FIELD_NUMBER',
 'INSPECTION_NUMBER',
 'NAICS_CODE',
 'OFFICE_ID',
 'SAMPLE_RESULT',
 'SAMPLING_NUMBER',
 'SIC_CODE',
 'STATE',
 'SUBSTANCE',
 'YEAR',
 'ZIP_CODE']

In [ ]:
where_diff = diff_columns['DATE_REPORTED']